# Image ==> Matrix
Given Image Address in Share Directory, Generate Geometry Output as Matrices. \
Split into binary geometry matrix and RI matrix, geometry matrix for meshing.

In [17]:
# Import libraries:
import numpy as np
from PIL import Image 
import tiffile as tf
import scipy as sp
import os

## Set-up I/O Path Allocations 
Declare input data path and access tiff or tif. \
The path variable can be changed to output different datas' geometry arrays.

In [ ]:
# Path Variables:
repo_path = "/Users/caozehuang/Documents/GitHub/Photon-Transport-Simulation/"     # Replace with repository path
geo_subfolder_address = "data/processed_geometry/"
RI_subfolder_address = "data/processed_RI/"
raw_subfolder_address = "data/raw/"

# Helpers:
def output_address(file_name, appendent="Raw", subfolder_address="data/", repo_path=repo_path):
    base_name = file_name.split(".")[0]
    out_path = f"{repo_path}{subfolder_address}{base_name}_{appendent}.mat"
    return out_path

# Declarables: 
file_name = "Reslice of 37_HT3D-1-1.tif"     # Replace with actual file name

access_path = f"{repo_path}{raw_subfolder_address}{file_name}"
print("Accessing: " + access_path)

Accessing: /Users/caozehuang/Documents/GitHub/Photon-Transport-Simulation/data/raw/Reslice of 37_HT3D-1-1.tif


In [19]:
# Load raw file:
if(file_name.split(".")[-1] == "tiff" or file_name.split(".")[-1] == "tif"):
    raw = tf.imread(access_path)
    print(f"TIFF loaded successfully!")
elif(file_name.split(".")[-1] == "jpeg" or file_name.split(".")[-1] == "jpg" or file_name.split(".")[-1] == "png"):
    raw = Image.open(access_path)
    print(f"Image loaded successfully!")
else:
    raise ValueError("Unsupported file format. Please use TIFF, JPEG, or PNG.")

TIFF loaded successfully!


In [20]:
# Convert to numpy array (already a numpy matrix)
try:
    raw_mat = np.array(raw)
except Exception as e:
    print(f"Error converting image to numpy array: {e}")
    raise

# Print raw file information:
print(f"Shape: {raw_mat.shape}")
print(f"Data type: {raw_mat.dtype}")
print(f"Min value: {raw_mat.min()}, Max value: {raw_mat.max()}")
print(raw_mat)  # Print the raw matrix to verify its contents

Shape: (212, 376)
Data type: uint16
Min value: 13244, Max value: 14041
[[13325 13325 13324 ... 13326 13326 13326]
 [13323 13320 13320 ... 13325 13326 13325]
 [13331 13329 13324 ... 13324 13325 13328]
 ...
 [13331 13333 13331 ... 13324 13325 13328]
 [13342 13339 13330 ... 13322 13325 13334]
 [13336 13335 13328 ... 13325 13326 13331]]


In [21]:
# Fix RI values of raw Fiji Output to be in the range of 1.0 to 2.0:
ri_mat = raw_mat / 10000
print(f"Shape: {ri_mat.shape}")
print(f"Data type: {ri_mat.dtype}")
print(f"Min value: {ri_mat.min()}, Max value: {ri_mat.max()}")
print(ri_mat)  # Print the RI matrix to verify its contents

Shape: (212, 376)
Data type: float64
Min value: 1.3244, Max value: 1.4041
[[1.3325 1.3325 1.3324 ... 1.3326 1.3326 1.3326]
 [1.3323 1.332  1.332  ... 1.3325 1.3326 1.3325]
 [1.3331 1.3329 1.3324 ... 1.3324 1.3325 1.3328]
 ...
 [1.3331 1.3333 1.3331 ... 1.3324 1.3325 1.3328]
 [1.3342 1.3339 1.333  ... 1.3322 1.3325 1.3334]
 [1.3336 1.3335 1.3328 ... 1.3325 1.3326 1.3331]]


# Extraction and Processing
Move to process the raw tiff file, return binary mask of matrix and save the work in allocated output location.

In [23]:
# Apply RI Threshold to mask the image:
ri_threshold = 1.34     # Can be changed

# Create binary matrix based on RI threshold
binary_mat = np.where(raw_mat >= ri_threshold, 1, 0)
print(f"Binary matrix created with threshold {ri_threshold}.\n")
print(f"Shape: {binary_mat.shape}\n")
print(f"Unique values in binary matrix: {np.unique(binary_mat)}\n")
print(binary_mat) # Print the binary matrix to verify its contents

Binary matrix created with threshold 1.34.

Shape: (212, 376)

Unique values in binary matrix: [1]

[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


# Saving

In [ ]:
# Save the binary matrix as a .mat file:
bin_output_path = output_address(file_name, appendent="Binary", subfolder_address=geo_subfolder_address)

# Save the RI matrix as a .mat file:
ri_output_path = output_address(file_name, appendent="RI", subfolder_address=RI_subfolder_address)
